In [ ]:
%matplotlib inline
import os, sys
import datetime as dt 
import yaml
import pandas as pd
import pcse
import pickle
from pcse.models import Wofost72_PP
from pcse.base import ParameterProvider
from pcse.exceptions import WeatherDataProviderError
from pcse.fileinput import ExcelWeatherDataProvider
import matplotlib.pyplot as plt


print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

# setting font and size 
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 16}
# change font
plt.rc('font', **font)

In [ ]:
# IND_10yrs = pd.DataFrame(wdp.export())
# IND_10yrs
# firstY = IND_10yrs[(IND_10yrs['DAY'] > dt.date(2013, 11, 1))& (IND_10yrs['DAY'] < dt.date(2014, 3, 1))]
# OUTPUT TO THE OUTSIDE FOR PROCESS
# firstY.to_excel('firstY_ind.xlsx')

In [ ]:
# prepared all artificial weather data in different periods
# rule is increase the TMAX by 5 degree in six different period 
# the interval for each period is 20 days 

import glob

# Use glob to find files that match the pattern
file_paths = glob.glob(f"Period*.xlsx")
# Read each Excel file and assign it to a unique variable name
for i, file_path in enumerate(file_paths):
    print(file_path)
    # Read the Excel file
    df = ExcelWeatherDataProvider(file_path)
    
    # Create a variable name based on a pattern (e.g., df1, df2, ...)
    variable_name = f'wdp{i + 1}'
    
    # Assign the dataframe to the variable name in the current environment
    globals()[variable_name] = df



In [ ]:
# load two data import functions from the pcse.fileinput class
from pcse.fileinput import YAMLCropDataProvider, CABOFileReader
# load the available crop list  
cropd = YAMLCropDataProvider(fpath='../data',
                              force_reload=True)  
# load the soil information
soild = CABOFileReader('../data/ec3 sandyloam.soil')
# load one data import function from the pcse.util class
from pcse.util import WOFOST72SiteDataProvider
# define the site initial condiations 
# WAV is the initial soil water content = 0; CO2 is the level of CO2 in the atmosphere = 360 ppm
sited = WOFOST72SiteDataProvider(WAV=10, CO2=360)
print(sited)
# help(WOFOST72SiteDataProvider)


In [ ]:
# Loop over crops, soils and years
agropotato = """
- 2013-11-02:
    CropCalendar:
        crop_name: potato
        variety_name: {Cultivar}
        crop_start_date: 2013-11-02
        crop_start_type: sowing
        crop_end_date: 2014-02-28
        crop_end_type: harvest
        max_duration: 300
    TimedEvents: null
    StateEvents: null
"""
cultivars = [
    "Fontane", 
    "Markies",
    "Premiere",
    "Festien", 
    "Innovator"]

In [ ]:
res = []

for c in cultivars:
    parameters = ParameterProvider(sitedata=sited, soildata=soild, cropdata=cropd)
    agromanagement = yaml.safe_load(agropotato.format(Cultivar = c))
    wofost =  Wofost72_PP(parameters, wdp1, agromanagement)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df0 = pd.DataFrame(output)
    df0['cultivar'] = c
    res.append(df0)
period0 = pd.concat(res)

res1 = []

for c in cultivars:
    parameters = ParameterProvider(sitedata=sited, soildata=soild, cropdata=cropd)
    agromanagement = yaml.safe_load(agropotato.format(Cultivar = c))
    wofost =  Wofost72_PP(parameters, wdp2, agromanagement)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df1 = pd.DataFrame(output)
    df1['cultivar'] = c
    res1.append(df1)
period1 = pd.concat(res1)

res2 = []

for c in cultivars:
    parameters = ParameterProvider(sitedata=sited, soildata=soild, cropdata=cropd)
    agromanagement = yaml.safe_load(agropotato.format(Cultivar = c))
    wofost =  Wofost72_PP(parameters, wdp3, agromanagement)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df1 = pd.DataFrame(output)
    df1['cultivar'] = c
    res2.append(df1)
period2 = pd.concat(res2)
res3 = []

for c in cultivars:
    parameters = ParameterProvider(sitedata=sited, soildata=soild, cropdata=cropd)
    agromanagement = yaml.safe_load(agropotato.format(Cultivar = c))
    wofost =  Wofost72_PP(parameters, wdp4, agromanagement)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df1 = pd.DataFrame(output)
    df1['cultivar'] = c
    res3.append(df1)
period3 = pd.concat(res3)
res6 = []

for c in cultivars:
    parameters = ParameterProvider(sitedata=sited, soildata=soild, cropdata=cropd)
    agromanagement = yaml.safe_load(agropotato.format(Cultivar = c))
    wofost =  Wofost72_PP(parameters, wdp7, agromanagement)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df1 = pd.DataFrame(output)
    df1['cultivar'] = c
    res6.append(df1)
period6 = pd.concat(res6)


In [ ]:
period0['day'] = pd.to_datetime(period0['day'])
period0.set_index('day', inplace = True)
period1['day'] = pd.to_datetime(period1['day'])
period1.set_index('day', inplace = True)
period2['day'] = pd.to_datetime(period2['day'])
period2.set_index('day', inplace = True)
period3['day'] = pd.to_datetime(period3['day'])
period3.set_index('day', inplace = True)
period6['day'] = pd.to_datetime(period6['day'])
period6.set_index('day', inplace = True)

In [ ]:
period0

In [ ]:

plt.figure(figsize=(8, 6))  # Adjust the figure size if needed
plt.plot(period0[period0['cultivar'] == 'Fontane'].index, period0[period0['cultivar'] == 'Fontane']['LAI'], label='NORMAL')
plt.plot(period1[period1['cultivar'] == 'Fontane'].index, period1[period1['cultivar'] == 'Fontane']['LAI'], label='PERIOD1')
plt.plot(period2[period2['cultivar'] == 'Fontane'].index, period2[period2['cultivar'] == 'Fontane']['LAI'], label='PERIOD2')
plt.plot(period3[period6['cultivar'] == 'Fontane'].index, period3[period3['cultivar'] == 'Fontane']['LAI'], label='PERIOD3')
plt.plot(period6[period6['cultivar'] == 'Fontane'].index, period6[period6['cultivar'] == 'Fontane']['LAI'], label='PERIOD6')

# Add labels and legend
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
# plt.title('Multiple Series Plot')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()

In [ ]:

fig, ax = plt.subplots()
# ax2 = ax.twinx()

df_fontane_wide2.iloc[0:200, :].plot( y=['TBchanges_2'], ax = ax)
df_fontane_wide.iloc[0:200, :].plot( y=['changes_10'], ls = '--', ax = ax)
fig.autofmt_xdate()
plt.title('LAI Sensitivity to TBASE and SPAN changes over time for 1 season')
plt.show()

quick thoughts:   
1. both base temperature and span have effects on LAI   
2. LAI is not sensitive to both parameters until late in the season under indian condtions. 
3. TBASE came in earlier than SPAN but SPAN wears off later   
4. TBASE seems more important than the SPAN 
5. WHY THERE ARE UP AND DOWNS IN THE TBASE changes? 



In [ ]:
# what about yield 
TBASE = df_res2[(df_res2['Cultivar'] == 'Fontane')&(df_res2['TWSO'] != 0 )].loc[:, ['TWSO','TBASE']]
SPAN = df_res[(df_res['Cultivar'] == 'Fontane')&(df_res['TWSO'] != 0 )].loc[:, ['TWSO','SPAN']]

In [ ]:
TBASE['year'], TBASE['m'], TBASE['d'] = TBASE.index.year, TBASE.index.month, TBASE.index.day
TB_yield = TBASE[(TBASE['m'] == 2) & (TBASE['d'] == 28)]
SPAN['year'], SPAN['m'], SPAN['d'] = SPAN.index.year, SPAN.index.month, SPAN.index.day
SPAN_yield = SPAN[(SPAN['m'] == 2) & (SPAN['d'] == 28)]


In [ ]:
# Get unique category values
categories = TB_yield['TBASE'].unique()

# Create a figure and axis
fig, ax = plt.subplots()

# Plot each category with a different color
for category in categories:
    category_data = TB_yield[TB_yield['TBASE'] == category]
    ax.scatter(category_data.index, category_data['TWSO'], label=category)

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('TBASE')
plt.legend(title='TBASE')

# Show the plot
plt.show()

In [ ]:
# Get unique category values
categories = SPAN_yield['SPAN'].unique()

# Create a figure and axis
fig, ax = plt.subplots()

# Plot each category with a different color
for category in categories:
    category_data = SPAN_yield[SPAN_yield['SPAN'] == category]
    ax.scatter(category_data.index, category_data['TWSO'], label=category)

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Dry matter content kg ha-1')
plt.legend(title='SPAN')

# Show the plot
plt.show()

# BUT WHICH ONE IS MORE INFLUENTIAL?  

In [ ]:
TB_yield.reset_index().merge(SPAN_yield.reset_index(),left_on=['day', 'year', 'm','d'], right_on=['day', 'year', 'm','d'])
# TB_yield
merged = TB_yield.pivot(columns = 'TBASE', values = "TWSO").merge(SPAN_yield.pivot(columns = 'SPAN', values = "TWSO"), left_index=True, right_index=True)
merged['TB_changes'] = (merged[5.0] -merged[3.0])/2
merged['SPAN_changes'] = (merged[45.0] -merged[35.0])/10
merged
 

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots()

# Plot each category with a different color
for col in merged.columns[6:8]:
    ax.scatter(merged.index, merged[col], label=col)

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Dry matter content kg ha-1')
plt.legend(title='changes per unit')
plt.title('absolute difference of per unit of parameter value change over time')
# Show the plot
plt.show()
merged.columns

In [ ]:
# what happened at year 2019 and 2022
weather =pd.DataFrame(  wdp.export())
weather.DAY = pd.to_datetime(weather.DAY)
weather.set_index('DAY', inplace = True)
weather['year'], weather['m'], weather['d'] = weather.index.year, weather.index.month, weather.index.day

In [ ]:
weather['TMAX'].plot()
weather['TMIN'].plot()


### Next step 
1. choose a cultivar - which one does not matter I think since we are not after the accuracy but the variation over the seasons. 
2. demonstrate the relationship between parameter values and output? LAI or twso? 

$$sX(t) = ∂Y(t)/∂X(t)$$

In [ ]:
import sympy as sp

In [ ]:
df_fontane2[0:1]

In [ ]:
dY_dX3 = sp.diff(df_fontane2.loc[df_fontane2['TBASE'] == 3, 'LAI'],3)
dY_dX4 = sp.diff(df_fontane2.loc[df_fontane2['TBASE'] == 4, 'LAI'],4)
dY_dX5 = sp.diff(df_fontane2.loc[df_fontane2['TBASE'] == 5, 'LAI'],5)

# Create a figure and axis
plt.figure(figsize=(8, 6))  # Adjust the figure size if needed
plt.plot(dY_dX3[dY_dX3 > 0].index, dY_dX3[dY_dX3 > 0], label='3')
plt.plot(dY_dX4[dY_dX4 > 0].index, dY_dX4[dY_dX4 > 0], label='4')
plt.plot(dY_dX5[dY_dX5 > 0].index, dY_dX5[dY_dX5 > 0], label='5')

# Add labels and legend
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
# plt.title('Multiple Series Plot')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()
# dY_dX[dY_dX > 0].plot()



In [ ]:
# Create a figure and axis
plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
plt.plot(dY_dX3[0:100][dY_dX3 > 0].index, dY_dX3[0:100][dY_dX3 > 0], label='3')
plt.plot(dY_dX4[0:100][dY_dX4 > 0].index, dY_dX4[0:100][dY_dX4 > 0], label='4')
plt.plot(dY_dX5[0:100][dY_dX5 > 0].index, dY_dX5[0:100][dY_dX5 > 0], label='5')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('rate of changes ')
# plt.title('Multiple Series Plot')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()

In [ ]:
dY_dX35 = sp.diff(df_fontane.loc[df_fontane['SPAN'] == 35, 'LAI'],35)
dY_dX40 = sp.diff(df_fontane.loc[df_fontane['SPAN'] == 40, 'LAI'],40)
dY_dX45 = sp.diff(df_fontane.loc[df_fontane['SPAN'] == 45, 'LAI'],45)

# Create a figure and axis
plt.figure(figsize=(8, 6))  # Adjust the figure size if needed
plt.plot(dY_dX35[dY_dX35 > 0].index, dY_dX35[dY_dX35 > 0], label='SPAN_35')
plt.plot(dY_dX40[dY_dX40 > 0].index, dY_dX40[dY_dX40 > 0], label='SPAN_40')
plt.plot(dY_dX45[dY_dX45 > 0].index, dY_dX45[dY_dX45 > 0], label='SPAN_45')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Rate of changes ')
# plt.title('Multiple Series Plot')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()


In [ ]:
dY_dX35 = sp.diff(df_fontane.loc[df_fontane['SPAN'] == 35, 'LAI'],35)
dY_dX40 = sp.diff(df_fontane.loc[df_fontane['SPAN'] == 40, 'LAI'],40)
dY_dX45 = sp.diff(df_fontane.loc[df_fontane['SPAN'] == 45, 'LAI'],45)

# Create a figure and axis
plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
plt.plot(dY_dX35[0:100][dY_dX35 > 0].index, dY_dX35[0:100][dY_dX35 > 0], label='SPAN_35')
plt.plot(dY_dX40[0:100][dY_dX40 > 0].index, dY_dX40[0:100][dY_dX40 > 0], label='SPAN_40')
plt.plot(dY_dX45[0:100][dY_dX45 > 0].index, dY_dX45[0:100][dY_dX45 > 0], label='SPAN_45')
plt.plot(dY_dX3[0:100][dY_dX3 > 0].index, dY_dX3[0:100][dY_dX3 > 0], label='3')
plt.plot(dY_dX4[0:100][dY_dX4 > 0].index, dY_dX4[0:100][dY_dX4 > 0], label='4')
plt.plot(dY_dX5[0:100][dY_dX5 > 0].index, dY_dX5[0:100][dY_dX5 > 0], label='5')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Rate of changes ')
# plt.title('Multiple Series Plot')
plt.legend()
plt.xticks(rotation=45)
# plt.autofmt_xdate()
# Show the plot
plt.grid(True)
plt.show()

In [ ]:
# Create a figure and axis
plt.figure(figsize=(10, 6))  # Adjust the figure size if needed
plt.plot(dY_dX35[0:100][dY_dX35 > 0].index, dY_dX35[0:100][dY_dX35 > 0], label='SPAN_35')
plt.plot(dY_dX40[0:100][dY_dX40 > 0].index, dY_dX40[0:100][dY_dX40 > 0], label='SPAN_40')
plt.plot(dY_dX45[0:100][dY_dX45 > 0].index, dY_dX45[0:100][dY_dX45 > 0], label='SPAN_45')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Rate of changes ')
# plt.title('Multiple Series Plot')
plt.legend()
# plt.autofmt_xdate()
# Show the plot
plt.grid(True)
plt.show()

In [ ]:
# Evaluate the derivative at a specific time t
t_value = 4  # Replace with the desired time value
sensitivity = dY_dX.subs(dY_dX.index, t_value)

# Print or use the sensitivity value
print("Sensitivity at time", t_value, ":", sensitivity)

In [ ]:
t, X, Y = sp.symbols('t X Y')


In [ ]:
Y_function = sp.sin(X*t)  # You can replace this with your actual function
dY_dX = sp.diff(Y_function, X)
dY_dX
t_value = 2.0  # Replace with the desired time value
sensitivity = dY_dX.subs(t, t_value)
print("Sensitivity at time", t_value, ":", sensitivity)


In [ ]:
dY_dX